## 探测异常值
由于**系统误差**，**人为误差**或者**固有数据的变异**使得他们与总体的行为特征，结构或相关性等不一样，这部分数据称为**异常值（outliner）**。
应用：异常值检测在数据挖掘中有着重要的意义，比如如果异常值是由于数据本身的变异造成的，那么对他们进行分析，就可以发现隐藏的更深层次的，潜在的，有价值的信息。例如发现金融和保险的欺诈行为，黑客入侵行为，还有就是追寻极低或者极高消费人群的消费行为，然后做出相对应的产品。

## 统计学方法对异常值的检测

#### 3σ探测方法
对于任何$\varepsilon > 0$，有$P(|X - E(X)| \geq \varepsilon) \leq \frac {D(X)}{(\varepsilon)^2}$

当$\varepsilon = 3\delta$时，统计数据与平均值的离散程度可以由其标准差$D(X) = \delta$反映。
+ 所有数据中，至少有3/4（或75%）的数据位于平均数2个标准差范围内。
+ 所有数据中，至少有8/9（或88.9%）的数据位于平均数3个标准差范围内。
+ 所有数据中，至少有24/25（或96%)的数据位于平均数5个标准差范围内。

所以，一般是把超过三个离散值的数据称之为异常值。

#### 散点图
可以使用pandas的`df.plot()`方法或seaborn的`sns.jointplot()`方法实现

#### 四分位展布法
把数据按照从小到大排序，其中25%处为上四分位（$F_L$），75%处为下四分位处（$F_U$）。则展布为$d_F = F_L - F_U$，上截断点为$F_L - 1.5d_F$，下截断点为$F_U + 1.5d_F$。通常将小于上截断点，或者大于下截断点的数据称为异常值。

四分位法可以通过箱形图表示，不过需要注意箱形图中数据值较大的位于上方，数据值较小的位于下方，应于四分位展布法中的描述加以区分。

优点：与方差和极差相比，更加不如意受极端值的影响，且处理大规模数据效果很好。

缺点：小规模处理略显粗糙。而且只适合单个属相的检测。

### 基于分布的异常值检测

#### 假设检验与显著性水平
假设检验是对于总体的一个普遍性论断。我们通常将一个要检验的假设记作$H_0$，称为**原假设（或零假设）（null hypothesis）**。当然，在研究中我们不希望否证自己的研究假设，因此通常将研究假设的对立假设作为原假设，而将我们自己的研究假设作为原假设的对立$H_1$，也就是**备选假设（alternative hypothesis）**。通过假设检验否定原假设，从而证明我们的研究假设是正确的。

这里有两个核心观点：
+ **不轻易拒绝原假设**。由于在实际中观察的样本数量较少，观察值存在一定波动。因此要给原假设一定范围的容忍度。这个容忍度就是**显著性水平$\alpha$**。
+ **小概率事件发生不正常**。如果超过容忍度的事件（也就是小概率事件），那么就说明原假设有问题，从而拒绝原假设。这也被称为**第一类错误**。

举例：

某药品商宣传能治愈某病的概率是90%。（即原假设）

一个医生不相信宣传，于是做个了实验验证，15个人治好了11个人。而15个人应该能够治愈13.5个人。那么宣传是不是骗人的呢？

这时候用**假设性检验**来验证(采用**显著性水平为5%**检验)，假设这15个人服从二项分布，P(X<=11)的概率等于5.6%，这个**p值大于显著性水平5%**。而我们的显著性水平是5%，也就是说，小于5%的是个小概率事件，治愈了11个人并不是一个小概率事件，在治愈率90%的情况下你是有可能刚好抽到治愈11个人的情况。我们没有足够的证据证明药品商是骗人的，所以我们接受他的宣传（**接受原假设**），即治愈率90%。

这时候有人会问，如果15个人治愈了9个人呢，我们经过计算发现，p值小于5%，这时候处于**拒绝域**。因为，你们宣传治愈率90%，可是我做了抽样，发现15个人只治好了9个人，概率太小了，基本不可能遇到的情况**（小概率事件）**怎么刚好让我遇到？所以，我们有足够的证据证明宣传是假的。这时候我们采用**备选假设**，**拒绝零假设**。

#### 利用假设检验异常值的思路
假设数据分布或者概率满足零假设$H_0$，但如果某个数据点的值拒绝零假设而接受备选假设$H_1$，那么就可以认为这个数据点与总体分布不符合，是一个异常值。

### 基于分布的异常值检测方法
基于分布的异常值检测方法通常只适用于单维数据，且必须确定其数据分布（通常是正态分布）。

#### Grubbs检验
1. 先把数据按照值从小到大的顺序排列，$x_1,x_2,x_3,...,x_n$
2. 计算均值$\overline{x}$和标准差$s$
3. 假设我们认为$x_i$为可疑点（一般取最小的或最大的某几个点作为可疑点），计算统计量：$g_i = \frac{|x_i - \overline{x}|}{s}$
4. 将$g_i$与查Grubbs临界值表所得的$g(a,n)$进行比较。如果$g_i < g(a,n)$,那么则认为该可疑点不是异常值，如果$g_i < g(a,n)$，就认为该可疑点是异常值
5. 重复以上步骤，直至所有的可疑点检测完毕

#### Dixon检测
1. 先将数据按照从小到大的顺序排列，$x_1,x_2,x_3,...,x_n$
2. 计算$r_大$和$r_小$值

    $r_大 = \frac {x_n - x_{n-1}}{x_n - x_1}$

    $r_小 = \frac {x_2 - x_1}{x_n - x_1}$

3. 将$r_大$与$r_小$Dixon系数表的临界值$r(a, n)$进行比较，如果$r_大 > r(a, n)$，则最大值为异常值，如果$r_小 > r(a, n)$则最小值为异常值。 

#### 3t分布检验

### 基于距离的异常值检测
基于距离的方法是基于下面这个假设：即若一个数据对象和大多数点距离都很远，那这个对象就是异常。
基于距离的方法比基于统计类方法要简单得多；因为为一个数据集合定义一个距离的度量要比确定数据集合的分布容易的多。

该方法通常要为数据对象定义一个异常分，最简单的定义方法是用**数据对象与最近的K个点的距离之和**。很明显，与K个最近点的距离之和越小，异常分越低；与K个最近点的距离之和越大，异常分越大。我们设定一个距离的阈值，异常分高于这个阈值，对应的数据对象就是异常点。

基于距离的方法简单易用。但也有缺点，比如计算复杂度是$O(m^2)$,m为样本数；而且方法对参数K和阈值敏感，而且，对于密度不均匀的点，该方法通常会给出错误的结果。

### 基于密度的异常值检测
详见[异常检测之基于密度的方法](https://zhuanlan.zhihu.com/p/26525803)

### 基于分类模型的异常值检测
根据已有的数据建立模型，然后对新来的数据点进行判断，从而认定其是否与整体偏离，如果偏离则为异常值。

常用的模型有贝叶斯模型、神经网络、决策树分类、SVM等。